In [ ]:
from pinecone import Pinecone
from transformers import BertTokenizerFast  # !pip install transformers

# load bert tokenizer from huggingface
tokenizer = BertTokenizerFast.from_pretrained(
   'bert-base-uncased'
)

In [ ]:
contexts = ["1231", "sfafs"]

In [ ]:
inputs = tokenizer(
   contexts[0], padding=True, truncation=True,
   max_length=512
)
inputs.keys()

In [ ]:
input_ids = inputs["input_ids"]

In [ ]:
from collections import Counter

# convert the input_ids list to a dictionary of key to frequency values
sparse_vec = dict(Counter(input_ids))
sparse_vec

In [ ]:
# !pip install sentence-transformers
from sentence_transformers import SentenceTransformer

# load a sentence transformer model from huggingface
model = SentenceTransformer(
   'multi-qa-MiniLM-L6-cos-v1'
)

emb = model.encode(contexts[0])
emb.shape


In [ ]:
from pinecone import Pinecone, ServerlessSpec # !pip install pinecone-client

pinecone = Pinecone(
   api_key="pcsk_5KSJb6_S2hK46wjboLTgkAhmFizEcpZdk1X6tTd2N66vA7v1BaMUNZYCbgSv4t4dM9Jrkz",  # app.pinecone.io
)
# choose a name for your index
index_name = "hybrid-search-intro"

cloud = 'aws'
region = 'us-east-1'

# create the index
pinecone.create_index(
   name = index_name,
   dimension = 384,  # dimensionality of dense model
   metric = "dotproduct",
    spec=ServerlessSpec(
        cloud=cloud,
        region=region
    )
)


In [ ]:
def build_dict(input_batch):
 # store a batch of sparse embeddings
   sparse_emb = []
   # iterate through input batch
   for token_ids in input_batch:
       indices = []
       values = []
       # convert the input_ids list to a dictionary of key to frequency values
       d = dict(Counter(token_ids))
       for idx in d:
            indices.append(idx)
            values.append(d[idx])
       sparse_emb.append({'indices': indices, 'values': values})
   # return sparse_emb list
   return sparse_emb


def generate_sparse_vectors(context_batch):
    # create batch of input_ids
    inputs = tokenizer(
           context_batch, padding=True,
           truncation=True,
           max_length=512
    )['input_ids']
    # create sparse dictionaries
    sparse_embeds = build_dict(inputs)
    return sparse_embeds


In [ ]:
from tqdm.auto import tqdm

batch_size = 32

for i in tqdm(range(0, len(contexts), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(contexts))
    # extract batch
    context_batch = contexts[i:i_end]
    # create unique IDs
    ids = [str(x) for x in range(i, i_end)]
    # add context passages as metadata
    meta = [{'context': context} for context in context_batch]
    # create dense vectors
    dense_embeds = model.encode(context_batch).tolist()
    # create sparse vectors
    sparse_embeds = generate_sparse_vectors(context_batch)

    vectors = []
    # loop through the data and create dictionaries for upserts
    for _id, sparse, dense, metadata in zip(
        ids, sparse_embeds, dense_embeds, meta
    ):
        vectors.append({
            'id': _id,
            'sparse_values': sparse,
            'values': dense,
            'metadata': metadata
        })

    # upload the documents to the new hybrid index
    pinecone.upsert(vectors)

# show index description after uploading the documents
pinecone.describe_index_stats()
